In [ ]:
"""
Running Classification Script with Spectrograms, but on Google Colab with GPU acceleration. 
============================================

This script performs running pattern classification using spectrograms generated
from sensor data. It supports both ResNet50 and LSTM-based models.

Key Functions:
-------------
1. Model Setup:
   - Configurable model architectures (ResNet50, LSTM)
   - Customizable hyperparameters
   - Support for transfer learning and fine-tuning

2. Data Loading:
   - Loads TFRecord files containing spectrogram data
   - Handles both treadmill and overground data
   - Supports multiple subjects and running speeds

3. Training Pipeline:
   - Implements model training with early stopping
   - Saves model weights and training history
   - Provides validation metrics and confusion matrices

Directory Configuration:
----------------------
Before running this script, update these paths in the configuration section:
1. dir_root: Root directory of your project
2. dir_data: Location of prepared data
3. dir_results: Where to save model outputs

Example Directory Structure:
    project_root/
    ├── Data/
    │   ├── Prepared/
    │   │   └── tfrecords/
    │   └── Results/
    │       ├── model_weights/
    │       └── model_history/
    └── Code/
        └── functions_my_model_resnet.py

Configuration Parameters:
----------------------
- Model Architecture:
  - ResNet50 or LSTM options
  - Trainable/frozen layers
  - Dropout rates
- Training Parameters:
  - Batch size, epochs
  - Learning rate
  - Early stopping conditions
- Data Parameters:
  - Input shapes
  - Number of classes
  - Train/val split ratio

Required Dependencies:
--------------------
- tensorflow/keras: Deep learning framework
- numpy: Numerical operations
- sklearn: Metrics calculation
- matplotlib: Visualization
- Custom modules:
  - functions_classification_general
  - script_running_classification_spectrogram_v2
  - functions_my_model_resnet
  - functions_recurrent_model


  Parameters:
    -----------
    subjects : list
        List of subject IDs to process
    dir_root : str
        Root directory path
    model_name : str
        Name for saving model and results
    weights_to_use : str or None
        Path to pre-trained weights if using transfer learning
    val_split : float
        Validation split ratio (0-1)
    epochs : int
        Number of training epochs
    batch_size : int
        Batch size for training
    dropout : float
        Dropout rate (0-1)
    early_stopping_patience : int
        Number of epochs to wait before early stopping
    early_stopping_min_delta : float
        Minimum change in monitored quantity for early stopping
    input_my_model : keras.Input
        Input shape for the model
    input_resnet : keras.Input
        Input shape for ResNet (if used)
    which_spectrograms : str
        Directory name containing spectrogram data
    resnet_trainable : bool
        Whether to train ResNet layers
    n_bins : int
        Number of classification bins
    classification : bool
        Whether to perform classification (vs regression)
    flag_shuffle_files : bool
        Whether to shuffle input files
    model_to_use : str
        Model architecture to use ('resnet50_class' or 'lstm_model_class')
    layers_nodes : list
        List of node counts for LSTM layers
    flag_subject_id_classification : bool
        Whether to classify subject IDs
    test_speed : float
        Speed to use for testing
    learning_rate : float
        Learning rate for optimization
    flag_speed_classification : bool
        Whether to classify speeds

Author: Patrick Mayerhofer



"""


import os
import datetime
import keras
import tensorflow as tf
from google.colab import drive 

# Mount Google Drive (Uncomment if using Google Colab)
# drive.mount('/content/drive') 

# ---- USER CONFIGURATION ---- #
# Define root directory (User must modify this according to their environment)
dir_root = os.path.abspath("./")  # Default: current working directory

dir_data = os.path.join(dir_root, "Data/Prepared/tfrecords/")  # Path to data
dir_results = os.path.join(dir_root, "Results/")  # Path to store model outputs

# Ensure directories exist
os.makedirs(dir_data, exist_ok=True)
os.makedirs(dir_results, exist_ok=True)

# Check if GPU is available
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    print('GPU device not found')
else:
    print(f'Found GPU at: {device_name}')

# ---- MODEL CONFIGURATION ---- #
epochs = 5000
subjects = [34, 35, 36, 37, 38, 40, 41, 42, 43, 45, 46, 48, 49, 52, 53, 54, 55, 58, 59, 60, 61, 62, 63, 66, 67, 68, 69, 70, 72, 73, 74, 77, 79, 80, 82, 84, 85, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 98, 99, 100, 101, 102, 104, 105, 106, 107, 108, 110, 111, 112, 113, 114, 115, 116, 118, 119, 120, 122, 123, 125, 126, 127, 128, 130, 131, 132, 133, 135, 138, 139, 140, 142, 143, 146, 147, 150, 151, 154, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 173, 174, 176, 177, 179, 180, 182, 184, 185, 186, 188]
val_split = 0.2
batch_size = 32
model_to_use = 'resnet50_class'  # Options: 'resnet50_class', 'lstm_model_class'
weights_to_use = None

dropout = 0
layers_nodes = [16]
early_stopping_patience = 60
early_stopping_min_delta = 0

input_my_model = keras.Input(shape=(126, 40, 12, 1))
input_resnet = keras.Input(shape=(126, 40, 3))

resnet_trainable = True

data_name = "10k_250_0_bins2_onespeed"
which_spectrograms = os.path.join(dir_data, data_name, "Treadmill/")
model_name = os.path.join(dir_results, data_name + '_notransfer_nodrop')

classification = True  # Set to False for regression
n_bins = 2
flag_shuffle_files = False

# ---- IMPORT CUSTOM MODULES ---- #
# Ensure these scripts are available in the same directory or in an accessible module path
try:
    import script_running_classification_spectrogram_v2 as script
except ModuleNotFoundError:
    raise ImportError("Missing 'script_running_classification_spectrogram_v2'. Ensure it is in the same directory or installed as a module.")

# ---- RUN CLASSIFICATION SCRIPT ---- #
evaluated_train_loss, evaluated_val_loss = script.script_running_classification_spectrogram_v2(
    subjects, dir_root, model_name, weights_to_use,
    val_split, epochs, batch_size, dropout,
    early_stopping_patience, early_stopping_min_delta,
    input_my_model, input_resnet, which_spectrograms,
    resnet_trainable, n_bins, classification, flag_shuffle_files,
    model_to_use, layers_nodes
)

Mounted at /content/drive
/content/drive/My Drive/Running Plantiga Project/Code
Found GPU at: /device:GPU:0
/content/drive/My Drive/Running Plantiga Project
filenames:
['/content/drive/My Drive/Running Plantiga Project/Data/Prepared/tfrecords/10k_250_0_bins2_onespeed/Treadmill/SENSOR034.tfrecords', '/content/drive/My Drive/Running Plantiga Project/Data/Prepared/tfrecords/10k_250_0_bins2_onespeed/Treadmill/SENSOR035.tfrecords', '/content/drive/My Drive/Running Plantiga Project/Data/Prepared/tfrecords/10k_250_0_bins2_onespeed/Treadmill/SENSOR036.tfrecords', '/content/drive/My Drive/Running Plantiga Project/Data/Prepared/tfrecords/10k_250_0_bins2_onespeed/Treadmill/SENSOR037.tfrecords', '/content/drive/My Drive/Running Plantiga Project/Data/Prepared/tfrecords/10k_250_0_bins2_onespeed/Treadmill/SENSOR038.tfrecords', '/content/drive/My Drive/Running Plantiga Project/Data/Prepared/tfrecords/10k_250_0_bins2_onespeed/Treadmill/SENSOR040.tfrecords', '/content/drive/My Drive/Running Plantiga Pro